In [1]:
using LinearAlgebra
using Statistics

#= Canonical LP Conversion
@desc:
Takes a linear program as a mutable structure and returns its canonical
@param {mutable struct}
@return {mutable struct}
=#

function to_canonical(system)
    
    dims = size(system.mat);
    ineq = findall(x->x!="=",system.relation);

    if isempty(ineq) == false
        
        # If there are inequalities, for each row, append a standard vector; For each row, add a slack variable. Adjust x,c 
        
        for i in ineq
            
            slack = zeros(dims[1],1);
            
            if system.relation[i] == "<="
                slack[i,1] = 1;
            else
                slack[i,1] = -1;
            end
            
            system.mat = [system.mat slack];
            system.relation[i] = "=";
            system.sol = [system.sol; 0];
            system.obj = [system.obj; 0];
        end
    end
    
    if system.objtype != "min"
        
        # If the objective is not a minimization, multiply it by neg one and change objective type to maximum.
        
        system.obj = -system.obj;
        system.objtype = "min";
    end
    
    return system
end 

#= Phase II
@desc:
Given a feasible solution and a basis, solves for the optimal solution of an LP
@param {mutable struct} , {basis}
@return {mutable struct}
=#

function phase2(system,B,tol)
    
    if abs(prod(diag(lu(system.mat[:,B]).U))) < tol
        print("Singular Matrix\n")
        return system
    end

    U = inv(system.mat[:,B]);
    system.y = transpose(system.obj[B]) * U;
    case1 = transpose(system.y * system.mat - transpose(system.obj));
    
    #Calculate yA<=c
    
    infeasible = findall(x->x>tol,case1);
    if isempty(infeasible)
        print("Optimal Solution Found\n");
        return system;
    end
    new_basis = infeasible[1];
    
    #Check for infeasible rows. If none exist, x,y are optimal. Otherwise, select
    #any index and let that value be a new basis vector

    t = U * system.mat[:,new_basis];
    post = findall(x->x>tol,t);
    if isempty(post)
        print("No Optimal Solution\n");
        return system
    end
    
    #Find positive t. If none exist, objective can be drive to -inf.
    
    lambda = system.sol[B[post]] ./ t[post];
    lambda_star = minimum(lambda)[1];
    p = findall(x->x==lambda_star,lambda)[1];
    p = post[p];
    
    #Calculate lambda star and get the index of the respective basis vector
    
    system.sol[B[p]] = 0;
    B[p] = new_basis;
    system.sol[new_basis] = lambda_star;
    system.sol[setdiff(B,B[p])] = system.sol[setdiff(B,B[p])] - lambda_star * t[setdiff(1:length(t),p)];
    
    #Swap the old basis vector for the new basis vector. Calculate new x.

    phase2(system,B,tol)
    
    #Since this is a finite process, recurse. Although, cycling can happen with degenerate matrices.
end



function phase1(system,tol)
    
    negs = findall(x->x<0,system.rhs);
    if !isempty(negs)
        system.rhs[negs] = -system.rhs[negs];
        system.mat[negs,:] = -system.rhs[negs,:];
    end
    
    #Checks for negative values in b and swaps lhs for rhs if needed.
    
    nonpos = findall(x->x==0,system.rhs);
    if !isempty(nonpos)
        
        system.rhs[nonpos] = system.rhs[nonpos] + rand(length(nonpos)) * tol;
    end
    
    #If b has nonpos, pertubate slightly.
    
    dims = size(system.mat);
    system.sol = [zeros(Float64,dims[2]); system.rhs];
    system.obj = [zeros(Float64,dims[2]);ones(Float64,dims[1])];
    system.mat = [system.mat I];
    
    #This introduces a BFS z = b and min = sum(z)
    
    B = findall(x->x>0,system.sol);

    system = phase2(system,B,tol);
    
    #phase2 the new system and attain BOS, feasible solution for orignal system
    
    if transpose(system.obj) * system.sol > 0
        print("No feasible solution")
        return [system,B]
    end
    
    #if sum(z) > 0, then the system does not have a feasible solution
    
    system.mat = osys.mat;
    dims = size(system.mat);
    system.sol = system.sol[1:dims[2]];
    system.obj = osys.obj;
    system.objtype = osys.objtype;
    system.relation = osys.relation;
    
    #return system to its canonical except for the feasible solution
    
    B = findall(x->x>0,system.sol);
    
    #solve the system
    
    return [system,B]
end
    

phase1 (generic function with 1 method)

In [3]:
#=A = A[5:8,:]
#A = A + rand(8,17);
#obj = [0.0,16.0,0.0,10.0]
#obj = [7,4,0,40]
obj = [.212,.166,.209,.432,.452,.357,.524,.063,.171,.535,.675,.495,.643,0,.580,.603,.8]
#objtype = "max"
objtype = "min"
#relations0 = ["=","=","<="]
#relations0 = ["=","=","<=",">="]
#relations = ["=","=","<="]
#relations = ["=","=","<=",">="]
relation = [">=","<=",">=","<=",">=","<=",">=","<="]
rhs = [1985239000 * .8 ,1985239000 * .9,0.0,0,0,0,0.0,0]
#rhs = rand(4)
sol = zeros(Float64,17)
#sol = zeros(Float64,4)
y = zeros(Float64,8)
#y = zeros(Float64,4)=#

mutable struct Lsys
    mat
    sol
    rhs
    relation
    obj
    objtype
    y
end

In [4]:
osys = to_canonical(Lsys(copy(A),copy(sol),copy(rhs),deepcopy(relation),copy(obj),deepcopy(objtype),copy(y)));
lsys = Lsys(A,sol,rhs,relation,obj,objtype,y);

In [5]:
lsys = to_canonical(lsys);

In [6]:
(lsys,B) = phase1(lsys,10^-5);

Optimal Solution Found


In [7]:
lsys = phase2(lsys,B,10^-5);

Optimal Solution Found
